In [9]:
import argparse
import pickle
import re
from datetime import datetime, timedelta
from itertools import chain, tee
from os import environ, makedirs, path
from localtileserver import get_leaflet_tile_layer, TileClient

import eccodes
import pdbufr
import numpy as np
import pandas as pd
import xarray as xr
import branca.colormap as bc
import rasterio
import rioxarray
import ipyleaflet
from scipy.optimize import root_scalar
from scipy.spatial import KDTree
from matplotlib import pyplot

from tracks_utils import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Creating storm dataframe
df_storms = create_storms_df()
print(df_storms.stormIdentifier.unique())
df_storm = df_storms[df_storms.stormIdentifier == '10L']

['06L' '08L' '10L']


In [3]:
# Compute strike probability map and save it in raster file
strike_map_xr, tif_path = strike_probability_map(df_storm)

In [4]:
client = TileClient(tif_path)

In [5]:
def make_palette_dict(pal):
    perc = ["0-10%", "11-20%", "21-30%", "31-40%", "41-50%", "51-60%", "61-70%", "71-80%", "81-90%", "91-100%"]
    dictionary = {perc[0]:pal[0], perc[1]:pal[1], perc[2]:pal[2], perc[3]:pal[3], perc[4]:pal[4], perc[5]:pal[5], perc[6]:pal[6], perc[7]:pal[7], perc[8]:pal[8], perc[9]:pal[9]}
    return dictionary

In [23]:
palette = ["#c4ff70", "#6ae24c", "#2a9134", "#137547", "#054a29", "#2397d1", "#557ff3", "#143cdc", "#3910b4", "#1e0063"]
palette_dict = make_palette_dict(palette)
t = get_leaflet_tile_layer(client, name = "Strike Probability Map", opacity = 0.8, palette = palette, nodata=0.0)

In [24]:
# Create the legend control
# legend_control = ipyleaflet.LegendControl(palette_dict, name="Strike Probability", position="topright")

In [25]:
m = ipyleaflet.Map(center=(df_storm.latitude.iloc[0], df_storm.longitude.iloc[0]), zoom=3)
m.add_layer(t)
m.add_control(ipyleaflet.LayersControl())
m.add_control(ipyleaflet.FullScreenControl())
# m.add_control(legend_control)

In [26]:
with rasterio.open(tif_path) as r:
    minv = "%.2f" % round(r.read(1).ravel().min(), 1)
    maxv = "%.2f" % round(r.read(1).ravel().max(), 1)
    
cmap_control = ipyleaflet.ColormapControl(
                                caption = "Strike probability",
                                colormap = bc.StepColormap(palette),
                                value_min = float(minv),
                                value_max = float(maxv),
                                position = 'topright',
                                transparent_bg = True
                                )
m.add(cmap_control)
display(m)

0.0
100.0
0.00 100.00


Map(center=[31.6, -65.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

Below: code to add the colormap\
r: rasterio.open(tif_path)\
bc: branca.colormap\
palette: will work with the palette list you already created

In [ ]:
minv = "%.2f" % round(r.read(1).ravel().min(), 1)
maxv = "%.2f" % round(r.read(1).ravel().max(), 1)
cmap_control = ColormapControl(
                                caption = "Strike prob ....",
                                colormap = bc.StepColormap(palette),
                                value_min = float(minv),
                                value_max = float(maxv),
                                position = 'topright',
                                transparent_bg = True
                                )
m.add(cmap_control)
r.close() #close the raster dataset once plotted

In [9]:
# Palette pink-yellow-orange-red
# ["#f7edf0", "#f4cbc6", "#f4afab", "#f4eea9", "#f4f482", "#f79d65", "#f4845f", "#f27059", "#f25c54", "#88292f"]

In [10]:
# Palette green to blue
# ["#c4ff70", "#6ae24c", "#2a9134", "#137547", "#054a29", "#2397d1", "#557ff3", "#143cdc", "#3910b4", "#1e0063"]